In [3]:
from config import Config
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
cfg = Config.from_env()

In [ ]:
# Import main package first to trigger model_rebuild() for all 866 models
import fhirsdk

from fhirsdk.hl7_fhir_r6_core.bundle import Bundle, BundleEntry
from fhirsdk.hl7_fhir_r6_core.resource import Resource
from fhirsdk.hl7_fhir_r6_core.domain_resource import DomainResource

In [ ]:
from fhirsdk.client import Client, Auth, AuthCredentials

In [ ]:
client = Client(cfg.endpoint_url, auth=Auth(method="basic", credentials=AuthCredentials(username=cfg.fhir_username, password=cfg.fhir_password)))

In [ ]:
from fhirsdk.hl7_fhir_r6_core.patient import Patient   

patient_search = client.search(Patient)
print(len(patient_search.entry))

In [ ]:
initial = patient_search.entry[0]

In [ ]:
# initial.resource

_patient = client.read(Patient, initial.resource.id)
print(_patient)

In [ ]:
from fhirsdk import PlanDefinition
plan = client.search(PlanDefinition)

In [ ]:
len(plan.entry)

In [ ]:
for p in plan.entry:
    print(p.resource.id, p.resource.title)

In [ ]:
from fhirsdk import CarePlan

cp = client.search(CarePlan, {"patient": "Patient/00126e97-9e44-4c16-9afa-13b7dc01d356"})

In [ ]:
print(cp.total)

In [ ]:
for entry in plan.entry:
    print(entry.resource.id, entry.resource.title)
    _pd = client.read(PlanDefinition, entry.resource.id)
    print(_pd.id, _pd.title)

In [ ]:
from fhirsdk import ResearchStudy

study = client.read(ResearchStudy, "Dynamic-Schedule-Exit-Example")

In [ ]:
study.protocol

In [ ]:
from common import build_transition_graph, print_transition_graph

plan = client.read(PlanDefinition, "dynamic-visit-schedule-multiple-paths")

graph = build_transition_graph(plan)

In [ ]:
print_transition_graph(graph)

In [5]:
from enroll_patient import enrol_patient

research_subject = enrol_patient(
    research_study_id="Dynamic-Schedule-Exit-Example",
    patient_id="00126e97-9e44-4c16-9afa-13b7dc01d356",
    config=cfg,
    subject_id="subject-001",
)

In [7]:
from evaluate import evaluate

planned = evaluate(
    research_subject_id=research_subject.id, config=cfg)


=== Evaluation for Research Subject: faf14ce8-1833-4a4e-b7c7-845ff7e49e09 ===

Completed Actions (0):

Active Actions (0):

Available Next Actions (1):
  • 0700e721-1f12-4998-89b8-6f4e649b62f7: Screening

Recommended Next Actions (1):
  ⭐ 0700e721-1f12-4998-89b8-6f4e649b62f7: Screening
     PlanDefinition: dynamic-visit-schedule-exit-example-Screening-PlanDefinition


In [ ]:
from execute import execute
import datetime

executed = execute("faf14ce8-1833-4a4e-b7c7-845ff7e49e09", 
                   plan_definition_ids=["dynamic-visit-schedule-exit-example-Screening-PlanDefinition"], 
                   visit_date=(datetime.datetime.now() + datetime.timedelta(days=7)), 
                   config=cfg)

In [1]:
from fhirsdk import ResearchSubjectSubjectMilestone, CodeableConcept, Coding
import datetime

milestone = ResearchSubjectSubjectMilestone(
    date=datetime.date.today().isoformat(),
    milestone=[
        CodeableConcept(text="Subject Entered Into Trial", 
                        coding=[
                            Coding(
                                system="http://terminology.hl7.org/ValueSet/research-subject-milestones",
                                code="C161417",
                            )
                        ])
                        ]
    )

ResearchSubjectSubjectMilestone.model_validate(milestone)

ResearchSubjectSubjectMilestone(extension=None, id=None, modifier_extension=None, date='2026-01-12', milestone=[CodeableConcept(extension=None, id=None, coding=[Coding(extension=None, id=None, code='C161417', display=None, system='http://terminology.hl7.org/ValueSet/research-subject-milestones', user_selected=None, version=None)], text='Subject Entered Into Trial')], reason=None)